In [1]:
%cd /gxfs_work/cau/sunms534/trading_bot/

/gxfs_work/cau/sunms534/trading_bot


/gxfs_work/cau/sunms534/.conda/envs/my_pytorch_env/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import pandas as pd
from src.config import config 
from tqdm.notebook import tqdm
tqdm.pandas()

pd.set_option(
    'display.max_colwidth', 2000
)

## Experimental

In [3]:
news_csv = pd.read_csv(config.data.fnspid.raw)

/scratch/SlurmTMP/sunms534.10454573/ipykernel_1948800/1036305684.py:1: DtypeWarning: Columns (3,5,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  news_csv = pd.read_csv(config.data.fnspid.raw)


In [4]:
news_csv.rename(columns={
    "Date": "time",
    "Stock_symbol": "stocks",
    "Author": "author",
    "Article_title": "title",
    "Article":"body"
}, inplace=True)

In [5]:
N_0 = news_csv.shape[0]

In [6]:
news_csv = news_csv.loc[:, ['time', 'stocks', 'author', 'title', 'body']]

In [7]:
# Inspect rows with insufficient information 
news_csv[news_csv[['time', 'stocks', 'body']].isna().any(axis=1) & ~news_csv[['time', 'stocks', 'body']].isna().all(axis=1)].head()

,time,stocks,author,title,body
243305,2022-06-06 00:00:00 UTC,ARKK,NaN,Fox Business: Tom Lydon Tackles ETF Money In Motion,NaN
1409458,2022-06-06 00:00:00 UTC,KWEB,NaN,Fox Business: Tom Lydon Tackles ETF Money In Motion,NaN
1759129,2023-10-17 00:00:00 UTC,PFFL,NaN,Stocks Mostly Lower as Strong U.S. Economic Reports Push Bond Yields Higher,NaN
1902201,2023-12-16 20:00:00 UTC,RILYN,NaN,3 High-Yield Dividend Stocks to Buy Before the End of the Year,NaN
2020353,2022-06-06 00:00:00 UTC,SPY,NaN,Fox Business: Tom Lydon Tackles ETF Money In Motion,NaN


In [8]:
news_csv.dropna(subset=['time', 'stocks', 'body'], inplace=True)

In [9]:
N_1 = news_csv.shape[0]

In [10]:
print(f"{N_0=}, {N_1=}")
print(f"Dropped {N_0 - N_1} entries due to NaNs.")

N_0=15549299, N_1=2491778
Dropped 13057521 entries due to NaNs.


In [11]:
news_csv['time'] = pd.to_datetime(news_csv['time'])

Many time stamps don't contain intra-day information. Hence we need to set the intra-day time of these entries to the end of the day and mark them as special rows.

In [12]:
news_csv['time'].dt.hour.value_counts().sort_index().head(5)

time
0    2465156
1       1073
2       1202
3       1007
4       1303
Name: count, dtype: int64

In [13]:
x = news_csv['time'].dt
no_intra_day_entries_mask = (x.hour == 0) & (x.minute == 0)

In [14]:
# Inspect entries which we will modify
# news_csv.loc[no_intra_day_entries_mask, :].head()

In [15]:
news_csv["intra_day_time"] = True
news_csv.loc[no_intra_day_entries_mask, "intra_day_time"] = False

In [16]:
# Transform date into time by offsetting by one day
news_csv.loc[no_intra_day_entries_mask, "time"] = news_csv.loc[no_intra_day_entries_mask, "time"] + pd.offsets.Day(1)

In [17]:
news_csv.to_parquet(config.data.fnspid.raw_html_parsed)